<a href="https://colab.research.google.com/github/samurai-architect/avocado/blob/master/DCF_Web_Site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colab-env --upgrade
!pip install quickfs
!pip install pyngrok
!pip install dash

  Preparing metadata (setup.py) ... done
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3805 sha256=ddd8fe1e7fdd8d1fe09546a3c162d2b28b5889e04190139218a20033be8683c7
  Stored in directory: /root/.cache/pip/wheels/ae/36/4f/466c2cd4db5d08f317893a920c4a0f58a81459ee3bdb136d35
Successfully built colab-env
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=54d739960705d6802eb3a32796123ec26dcc00d507840a673da52d1d05e6fcb8
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling 

In [2]:
import colab_env
import os
import pandas as pd
import plotly.express as px
import threading
from quickfs import QuickFS
from dash import Dash, html, dcc, callback, Output, State, Input, dash_table
from pyngrok import ngrok

Mounted at /content/gdrive


In [3]:
!ngrok authtoken $NGROK_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


If you want to organize your code into modules you need to update the path to import from Google Drive:

```
# import sys
sys.path.append('/content/gdrive/My Drive')
```
Or
```
# import sys
sys.path.append('/content/gdrive/My Drive/myproject/mylibrary')
```

In [54]:
import logging
from dash import Dash, html, dcc, callback, Output, State, Input, dash_table

def get_qfs_financials(symbol):
  logging.debug("entering get_qfs_financials() for symbol " + symbol)
  api_key = os.getenv("QFS_API_KEY")
  client = QuickFS(api_key)
  # get the full data for GOOGL
  logging.debug("calling qfs:get_data_full for symbol: " + symbol)
  qfs_data = pd.DataFrame(client.get_data_full(symbol=symbol))
  logging.debug("returned from qfs:get_data_full for symbol: " + symbol)
  return qfs_data

def get_qfs_transposed(qfs_data, time_period):
  logging.debug("entering get_qfs_transformed()")
  if time_period == "annual" or time_period == "quarterly":
    logging.debug("calling pop_other_qfs_metrics()")
    other_metrics = pop_other_qfs_metrics(qfs_data, time_period)
    logging.debug("returned from pop_other_qfs_metrics()")
    period = pd.DataFrame(qfs_data["financials"][time_period])
    first_column = period.columns.values[0]
    period.set_index(first_column,inplace=True)
    period_t = period.transpose()
    return period_t.rename(columns = {'index':first_column})

def pop_other_qfs_metrics(stock_data, time_period):
  other_metrics = {}
  if time_period == "annual" or time_period == "quarterly":
    other_metrics["preliminary"] = stock_data["financials"][time_period].pop("preliminary")
    other_metrics["gross_margin_median"] = stock_data["financials"][time_period].pop("gross_margin_median")
    other_metrics["pretax_margin_median"] = stock_data["financials"][time_period].pop("pretax_margin_median")
    other_metrics["operating_income_margin_median"] = stock_data["financials"][time_period].pop("operating_income_margin_median")
    other_metrics["fcf_margin_median"] = stock_data["financials"][time_period].pop("fcf_margin_median")
    other_metrics["roa_median"] = stock_data["financials"][time_period].pop("roa_median")
    other_metrics["roe_median"] = stock_data["financials"][time_period].pop("roe_median")
    other_metrics["roic_median"] = stock_data["financials"][time_period].pop("roic_median")
    other_metrics["assets_to_equity_median"] = stock_data["financials"][time_period].pop("assets_to_equity_median")
    other_metrics["debt_to_assets_median"] = stock_data["financials"][time_period].pop("debt_to_assets_median")
    other_metrics["debt_to_equity_median"] = stock_data["financials"][time_period].pop("debt_to_equity_median")
    other_metrics["roic_5yr_avg"] = stock_data["financials"][time_period].pop("roic_5yr_avg")
  return other_metrics

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("staring...")
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')
app = Dash(__name__)
port = 8050

my_output = html.Div()

app.layout = html.Div([
                        dcc.Input(id='stock-symbol-state', type="text", value='AAPL'),
                        html.Button('Submit!', id='submit-button-state'),
                        html.Br(),
                        my_output
              ])

@callback(
    Output(my_output, 'children'),
    Input('submit-button-state', 'n_clicks'),
    State('stock-symbol-state', 'value')
)
def update_output(n_clicks, symbol):
  logging.debug("entering the callback...")
  if n_clicks is None:
      raise PreventUpdate
  else:
      logging.debug('calling get_qfs_transposed from the callback...')
      transposed_data =  get_qfs_transposed(get_qfs_financials(symbol), "annual")
      logging.debug('returned from get_qfs_transposed...')

      return    dash_table.DataTable(
                          data=transposed_data.to_dict('records'),
                          columns=[{"name": i, "id": i} for i in transposed_data.columns],
                          id='3'
                        )

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

#app.config["BASE_URL"] = public_url

if __name__ == '__main__':
    # set up logging to file
    logfile = 'example.log'
    loglevel = logging.DEBUG
    logging.basicConfig(filename=logfile, filemode='a', level=loglevel)
    logging.info('\n-------------------- START --------------------')
    #threading.Thread(target=app.run(jupyter_server_url= public_url), kwargs={"use_reloader": False}).start()
    app.run(jupyter_server_url= public_url, debug=True)

DEBUG:root:staring...
INFO:pyngrok.ngrok:Opening tunnel named: http-8050-f567c302-0830-4d48-a64b-0cbc8f6c6de1
DEBUG:pyngrok.process:Removing stale process for "ngrok_path" /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok
DEBUG:pyngrok.process:ngrok process starting with PID: 31583
INFO:pyngrok.process.ngrok:t=2023-08-30T01:08:49+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2023-08-30T01:08:49+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2023-08-30T01:08:49+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2023-08-30T01:08:49+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
INFO:pyngrok.process.ngrok:t=2023-08-30T01:08:49+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=27b8d18e7fa1
INFO:pyngrok.process.ngrok:t=2023-08-30T01:08:49+0000 lvl=info msg="t

 * ngrok tunnel "https://f999-35-196-8-72.ngrok-free.app" -> "http://127.0.0.1:8050"


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8050
DEBUG:urllib3.connectionpool:http://127.0.0.1:8050 "GET /_alive_b0c7b2e0-2920-45be-ad58-2964122e70f8 HTTP/1.1" 200 5


<IPython.core.display.Javascript object>

In [55]:
!killall ngrok

INFO:pyngrok.process.ngrok:t=2023-08-30T01:11:49+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:pyngrok.process.ngrok:t=2023-08-30T01:11:49+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
